In [ ]:
############################# USING STUFF
using HDF5
using Plots
using Distributed
using Interact
pyplot()

In [ ]:
############################# INPUT STUFF
input_name="2D_sinus_005_low"

#filename=""

In [ ]:
############################# LOAD SAVE FILES
cd("..")
include("Initial_data/"*input_name*".jl")
file="./data/"*potname*"_R0($R0)_p0($p0).h5"
#file="MQC/data/"*filename*".h5"

In [ ]:
############################# HISTOGRAM STUFF
HISTO_RES=100;
mins=[-50;-50]
maxs=[50;50]

In [ ]:
E0=h5read(file,"E0");                                     #LOAD INITIAL ENERGY

for dyn in DYN_LIST
    println("Loading $dyn...")
    if dyn in CL_LIST
        str="""T_$(dyn),R_$(dyn),P_$(dyn)=CL_read("$file","$dyn")"""
    elseif dyn in MF_LIST
        str="""T_$(dyn),R_$(dyn),P_$(dyn),C_$(dyn)=MF_read("$file","$dyn")"""
    elseif dyn in SH_LIST
        str="""T_$(dyn),R_$dyn,P_$(dyn),C_$(dyn),AST_$(dyn)=SH_read("$file","$dyn")"""
    end
    eval(Meta.parse(str))
end

println("Finished loading, starting histograms...")

for dyn in DYN_LIST
    tic=time()
    println("$dyn histogram...")
    str="Rbase_$dyn,HR_$dyn=multi_d_histo(R_$dyn,$mins,$maxs,$HISTO_RES)"
    eval(Meta.parse(str))
    str="HR_$dyn=HR_$dyn./maximum(HR_$dyn[:,:,1])"
    eval(Meta.parse(str))
    println("Finished $dyn histogram after $(time()-tic) seconds!")
end

In [ ]:
HR_EH_ORIG=copy(HR_EH);
HR_FSSH_ORIG=copy(HR_FSSH);
HR_CM2_FSSH_ORIG=copy(HR_CM2_FSSH);
HR_CM3_FSSH_ORIG=copy(HR_CM3_FSSH);

In [ ]:
EH_tresh=0.001
for ix in 1:100
    for iy in 1:100
        if HR_EH[ix,iy,end]<EH_tresh
            HR_EH[ix,iy,end]=0
        end
    end
end

In [ ]:
FSSH_tresh=0.0015
for ix in 1:100
    for iy in 1:100
        if HR_FSSH[ix,iy,end]<FSSH_tresh
            HR_FSSH[ix,iy,end]=0
        end
    end
end

In [ ]:
CM2_FSSH_tresh=0.001
for ix in 1:100
    for iy in 1:100
        if HR_CM2_FSSH[ix,iy,end]<CM2_FSSH_tresh
            HR_CM2_FSSH[ix,iy,end]=0
        end
    end
end

In [ ]:
CM3_FSSH_tresh=0.0008
for ix in 1:100
    for iy in 1:100
        if HR_CM3_FSSH[ix,iy,end]<CM3_FSSH_tresh
            HR_CM3_FSSH[ix,iy,end]=0
        end
    end
end

In [ ]:
dyn=CM3_FSSH
plt_str="contour(Rbase_$(dyn)[1][:,101],Rbase_$(dyn)[2][:,101],HR_$(dyn)[:,:,101])"
    eval(Meta.parse(plt_str))
    xticks!([(2k+1)*pi for k in -7:7])
    yticks!([(2k+1)*pi for k in -7:7])

In [ ]:
@manipulate for dyn in DYN_LIST, ipt in 1:101
    plt_str="contour(Rbase_$(dyn)[1][:,$ipt],Rbase_$(dyn)[2][:,$ipt],HR_$(dyn)[:,:,$ipt])"
    eval(Meta.parse(plt_str))
    xticks!([(2k+1)*pi for k in -7:7])
    yticks!([(2k+1)*pi for k in -7:7])
end